In [ ]:
import pandas as pd
import numpy as np
import os
import shutil

In [ ]:
from sqlalchemy import create_engine
engine = create_engine(r'sqlite:///C:\Users\njones\Documents\addresses.sqlite', echo=False)

In [ ]:
header_list = [
    'ROW_NUM','INPUT_ADDRESS','MATCH','MATCH_TYPE','TIGER_OUTPUT_ADDRESS','LONLAT','TIGERLINE_ID','TIGERLINE_ID_SIDE'
]

#.loc[row_indexer,col_indexer]

In [ ]:
for root, dirs, files in os.walk(r"C:\Users\njones\Documents\Python Scripts\Census Geocoding\downloaded_csvs"):
    for filename in files: #[:2]:
        print(root+"\\"+filename)
        df = pd.read_csv(r"%s"%(root+"\\"+filename), names=header_list)
        matches_df = df.loc[df['MATCH']=='Match']
        non_matches_df = df.loc[(df['MATCH']=='No_Match')|(df['MATCH']=='Tie')]

        matches_df[['STREET_ADDRESS', 'CITY', 'STATE','ZIP_CODE']] = matches_df['INPUT_ADDRESS'].str.split(",", expand=True)[[0,1,2,3]]
        non_matches_df[['STREET_ADDRESS', 'CITY', 'STATE','ZIP_CODE']] = non_matches_df['INPUT_ADDRESS'].str.split(",", expand=True)[[0,1,2,3]]

        non_matches_df[['O_ADDRESS', 'O_CITY', 'O_STATE','O_ZIP_CODE']] = ""
        matches_df[['O_ADDRESS', 'O_CITY', 'O_STATE','O_ZIP_CODE']] = matches_df['TIGER_OUTPUT_ADDRESS'].str.split(",", expand=True)[[0,1,2,3]]

        
        matches_df[['LON', 'LAT']] = matches_df['LONLAT'].str.split(",", expand=True)
        
        matches_df[['filename']] = filename
        non_matches_df[['filename']] = filename
        
        # Trim all non-numeric cloumn values of extra characters
        for c in matches_df.select_dtypes(exclude=[np.number]).columns.tolist():
            matches_df[c] = matches_df[c].str.strip()
            
        for c in non_matches_df.select_dtypes(exclude=[np.number]).columns.tolist():
            non_matches_df[c] = non_matches_df[c].str.strip()    
            

        # Write dataframe to the SQLite database
        matches_df.to_sql('address_coords', con=engine, index=False, if_exists='append') 
        non_matches_df.to_sql('address_coords', con=engine, index=False, if_exists='append') 
        
        dest = shutil.move(
            root+"\\"+filename,
            "C:\\Users\\njones\\Documents\\Python Scripts\\Census Geocoding\\inserted_csvs\\"+filename
            ) 

In [ ]:
non_matches_df

In [ ]:
matches_df['TIGER_OUTPUT_ADDRESS'].str.split(",", expand=True)[[0,1,2,3]]

In [ ]:
#df.to_sql('answers', con=engine, index=False, if_exists='append') 
header_list = [
    'ROW_NUM','INPUT_ADDRESS','MATCH','MATCH_TYPE','TIGER_OUTPUT_ADDRESS','LONLAT','TIGERLINE_ID','TIGERLINE_ID_SIDE'
]
df = pd.read_csv(r"C:\Users\njones\Documents\Python Scripts\Census Geocoding\downloaded_csvs\output_103_d.csv", names=header_list)
df

In [ ]:
matches_df = df.loc[df['MATCH']=='Match']
non_matches_df = df.loc[(df['MATCH']=='No_Match')|(df['MATCH']=='Tie')]

matches_df[['STREET_ADDRESS', 'CITY', 'STATE','ZIP_CODE']] = matches_df['TIGER_OUTPUT_ADDRESS'].str.split(",", expand=True)
non_matches_df[['STREET_ADDRESS', 'CITY', 'STATE','ZIP_CODE']] = non_matches_df['INPUT_ADDRESS'].str.split(",", expand=True)

matches_df[['LON', 'LAT']] = matches_df['LONLAT'].str.split(",", expand=True)

matches_df
#matches_df[['STREET_ADDRESS', 'CITY', 'STATE','ZIP_CODE']] = matches_df['INPUT_ADDRESS'].str.split(",", expand=True)
#non_matches_df[['STREET_ADDRESS', 'CITY', 'STATE','ZIP_CODE']] = non_matches_df['TIGER_OUTPUT_ADDRESS'].str.split(",", expand=True)

In [ ]:
df[['STREET_ADDRESS', 'CITY', 'STATE','ZIP_CODE']] = df['TIGER_OUTPUT_ADDRESS'].str.split(",", expand=True)
df.loc[df['TIGER_OUTPUT_ADDRESS'].isna()][['STREET_ADDRESS', 'CITY', 'STATE','ZIP_CODE']] = df.loc[df['TIGER_OUTPUT_ADDRESS'].isna()]['INPUT_ADDRESS'].str.split(",", expand=True)

df[['LON', 'LAT']] = df['LONLAT'].str.split(",", expand=True)
df

In [ ]:
matches_df = df.loc[~df['TIGER_OUTPUT_ADDRESS'].isna()][['STREET_ADDRESS', 'CITY', 'STATE','ZIP_CODE']]
non_matches_df = df.loc[df['TIGER_OUTPUT_ADDRESS'].isna()][['STREET_ADDRESS', 'CITY', 'STATE','ZIP_CODE']]
matches_df[['STREET_ADDRESS', 'CITY', 'STATE','ZIP_CODE']] = matches_df['INPUT_ADDRESS'].str.split(",", expand=True)
non_matches_df[['STREET_ADDRESS', 'CITY', 'STATE','ZIP_CODE']] = non_matches_df['TIGER_OUTPUT_ADDRESS'].str.split(",", expand=True)

In [ ]:
df

In [ ]:
# Select non-numeric columns
df.select_dtypes(exclude=[np.number])

In [ ]:
for c in df.select_dtypes(exclude=[np.number]).columns.tolist():
    df[c] = df[c].str.strip()

In [ ]:
df

In [ ]:
query = """
        SELECT * FROM address_coords WHERE MATCH = 'Match' AND STATE = 'PA' ORDER BY ZIP_CODE
    """ #.format(cols,Q,segment)

pd.read_sql(query, engine).to_csv("PA_CUSTOMERS_WITH_GPS.csv")